In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


from sklearn import model_selection, preprocessing
import lightgbm as lgbm
import datetime
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output

# Any results you write to the current directory are saved as output.

train = pd.read_csv("../input/train.csv", parse_dates=['timestamp'])
test = pd.read_csv("../input/test.csv", parse_dates=['timestamp'])
df_macro = pd.read_csv("../input/macro.csv", parse_dates=['timestamp'])
id_test = test.id

In [ ]:
####################################################
d_train = pd.read_csv('../input/train.csv')
d_test = pd.read_csv('../input/test.csv')
id_test = test.id

mult = .969




##########
#convert data to in factorize
joined =pd.concat([d_train,d_test])

cat_cols = [x for x in joined.columns if joined[x].dtype == 'object']

for col in cat_cols:
   joined.loc[:,col] = pd.factorize(joined[col], sort=True)[0]

train = joined[joined['price_doc'].notnull()]
test = joined[joined['price_doc'].isnull()]

del joined

print(train.shape, test.shape)

y_train = train["price_doc"] * mult + 10
x_train = train.drop(["id", "timestamp", "price_doc"], axis=1)
x_test = test.drop(["id", "timestamp"], axis=1)
#########


'''
for c in x_train.columns:
    if x_train[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(x_train[c].values))
        x_train[c] = lbl.transform(list(x_train[c].values))
#This function I can't undersand


for c in x_test.columns:
    if x_train[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(x_train[c].values))
        x_train[c] = lbl.transform(list(x_train[c].values))
'''

predictions, model = runLGBM(x_train,y_train,x_test,seed_val=42)
output_2 = pd.DataFrame({'id': id_test, 'price_doc': predictions})
output_2.head()


# Any results you write to the cruuent directory are saved as output.
df_train = pd.read_csv("../input/train.csv", parse_dates=['timestamp'])
df_test = pd.read_csv("../input/test.csv", parse_dates=['timestamp'])
df_macro = pd.read_csv("../input/macro.csv", parse_dates=['timestamp'])

df_train.drop(df_train[df_train["life_sq"] > 7000].index, inplace=True)

y_train = df_train['price_doc'].values * mult + 10
id_test = df_test['id']


num_train = len(df_train)
df_all = pd.concat([df_train, df_test])
# Next line just adds a lot of NA columns (becuase "join" only works on indexes)
# but somewhow it seems to affect the result
df_all = df_all.join(df_macro, on='timestamp', rsuffix='_macro')
print(df_all.shape)

# Add month-year
month_year = (df_all.timestamp.dt.month + df_all.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
df_all['month_year_cnt'] = month_year.map(month_year_cnt_map)

# Add week-year count
week_year = (df_all.timestamp.dt.weekofyear + df_all.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
df_all['week_year_cnt'] = week_year.map(week_year_cnt_map)

# Add month and day-of-week
df_all['month'] = df_all.timestamp.dt.month
df_all['dow'] = df_all.timestamp.dt.dayofweek

# Other feature engineering
df_all['rel_floor'] = df_all['floor'] / df_all['max_floor'].astype(float)
df_all['rel_kitch_sq'] = df_all['kitch_sq'] / df_all['full_sq'].astype(float)
joined =pd.concat([train,test])

cat_cols = [x for x in joined.columns if joined[x].dtype == 'object']

for col in cat_cols:
   joined.loc[:,col] = pd.factorize(joined[col], sort=True)[0]

train = joined[joined['price_doc'].notnull()]
test = joined[joined['price_doc'].isnull()]

del joined

print(train.shape, test.shape)

#train_y = np.log(train["price_doc"])
train_y = train["price_doc"]
train_X = train.drop(["id", "timestamp", "price_doc"], axis=1)
test_X = test.drop(["id", "timestamp"], axis=1)

predictions, model = runLGBM(train_X, train_y, test_X, seed_val=42)


predictions = np.round(predictions * 0.99)
output_1 = pd.DataFrame({'id': id_test, 'price_doc': predictions})



In [ ]:
def runLGBM(train_X, train_y, test_X, seed_val=42):
    
    params = {
        'boosting_type': 'gbdt', 'objective': 'regression', 'nthread': -1, 'verbose': 0,
        'num_leaves': 31, 'learning_rate': 0.05, 'max_depth': -1,
        'subsample': 1, 'subsample_freq': 1, 'colsample_bytree': 0.6, 
        'reg_alpha': 1, 'reg_lambda': 0.001, 'metric': 'rmse',
        'min_split_gain': 0.5, 'min_child_weight': 1, 'min_child_samples': 10, 'scale_pos_weight': 1
        }
    #kf = KFold(n_splits=5, shuffle=True, random_state=seed_val)
    pred_test_y = np.zeros(test_X.shape[0])
    
    train_set = lgbm.Dataset(train_X, train_y, silent=True)
        
    #practical_model = lgbm.train(practical_model_params, train_set=train_set, num_boost_round=1000,evals_result=[(dval, 'val')], early_stopping_rounds=20, verbose_eval=20)


    model = lgbm.train(params, train_set=train_set, num_boost_round=300)
    
    pred_test_y = model.predict(test_X, num_iteration = model.best_iteration)
    #print(best_num)
    return pred_test_y , model

In [ ]:


joined =pd.concat([train,test])

cat_cols = [x for x in joined.columns if joined[x].dtype == 'object']

for col in cat_cols:
   joined.loc[:,col] = pd.factorize(joined[col], sort=True)[0]

train = joined[joined['price_doc'].notnull()]
test = joined[joined['price_doc'].isnull()]

del joined

print(train.shape, test.shape)

#train_y = np.log(train["price_doc"])
train_y = train["price_doc"]
train_X = train.drop(["id", "timestamp", "price_doc"], axis=1)
test_X = test.drop(["id", "timestamp"], axis=1)

predictions, model = runLGBM(train_X, train_y, test_X, seed_val=42)


predictions = np.round(predictions * 0.99)
output_1 = pd.DataFrame({'id': id_test, 'price_doc': predictions})



####################################################
d_train = pd.read_csv('../input/train.csv')
d_test = pd.read_csv('../input/test.csv')
id_test = test.id

mult = .969




##########
#convert data to in factorize
joined =pd.concat([d_train,d_test])

cat_cols = [x for x in joined.columns if joined[x].dtype == 'object']

for col in cat_cols:
   joined.loc[:,col] = pd.factorize(joined[col], sort=True)[0]

train = joined[joined['price_doc'].notnull()]
test = joined[joined['price_doc'].isnull()]

del joined

print(train.shape, test.shape)

y_train = train["price_doc"] * mult + 10
x_train = train.drop(["id", "timestamp", "price_doc"], axis=1)
x_test = test.drop(["id", "timestamp"], axis=1)
#########


'''
for c in x_train.columns:
    if x_train[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(x_train[c].values))
        x_train[c] = lbl.transform(list(x_train[c].values))
#This function I can't undersand


for c in x_test.columns:
    if x_train[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(x_train[c].values))
        x_train[c] = lbl.transform(list(x_train[c].values))
'''

predictions, model = runLGBM(x_train,y_train,x_test,seed_val=42)
output_2 = pd.DataFrame({'id': id_test, 'price_doc': predictions})
output_2.head()


# Any results you write to the cruuent directory are saved as output.
df_train = pd.read_csv("../input/train.csv", parse_dates=['timestamp'])
df_test = pd.read_csv("../input/test.csv", parse_dates=['timestamp'])
df_macro = pd.read_csv("../input/macro.csv", parse_dates=['timestamp'])

df_train.drop(df_train[df_train["life_sq"] > 7000].index, inplace=True)

y_train = df_train['price_doc'].values * mult + 10
id_test = df_test['id']


num_train = len(df_train)
df_all = pd.concat([df_train, df_test])
# Next line just adds a lot of NA columns (becuase "join" only works on indexes)
# but somewhow it seems to affect the result
df_all = df_all.join(df_macro, on='timestamp', rsuffix='_macro')
print(df_all.shape)

# Add month-year
month_year = (df_all.timestamp.dt.month + df_all.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
df_all['month_year_cnt'] = month_year.map(month_year_cnt_map)

# Add week-year count
week_year = (df_all.timestamp.dt.weekofyear + df_all.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
df_all['week_year_cnt'] = week_year.map(week_year_cnt_map)

# Add month and day-of-week
df_all['month'] = df_all.timestamp.dt.month
df_all['dow'] = df_all.timestamp.dt.dayofweek

# Other feature engineering
df_all['rel_floor'] = df_all['floor'] / df_all['max_floor'].astype(float)
df_all['rel_kitch_sq'] = df_all['kitch_sq'] / df_all['full_sq'].astype(float)


In [ ]:
'''
d_train['building_name'] = pd.factorize(d_train.sub_area + d_train['metro_km_avto'].astype(str))[0]
d_test['building_name'] = pd.factorize(d_test.sub_area + d_test['metro_km_avto'].astype(str))[0]

def add_time_features(col):
   col_month_year = pd.Series(pd.factorize(train[col].astype(str) + month_year.astype(str))[0])
   train[col + '_month_year_cnt'] = col_month_year.map(col_month_year.value_counts())

   col_week_year = pd.Series(pd.factorize(train[col].astype(str) + week_year.astype(str))[0])
   train[col + '_week_year_cnt'] = col_week_year.map(col_week_year.value_counts())
    
add_time_features('building_name')

def add_time_features(col):
   col_month_year = pd.Series(pd.factorize(test[col].astype(str) + month_year.astype(str))[0])
   test[col + '_month_year_cnt'] = col_month_year.map(col_month_year.value_counts())

   col_week_year = pd.Series(pd.factorize(test[col].astype(str) + week_year.astype(str))[0])
   test[col + '_week_year_cnt'] = col_week_year.map(col_week_year.value_counts())
    
add_time_features('building_name')
'''

# Remove timestamp column (may overfit the model in train)
df_all.drop(['timestamp', 'timestamp_macro'], axis=1, inplace=True)


factorize = lambda t: pd.factorize(t[1])[0]

df_obj = df_all.select_dtypes(include=['object'])

X_all = np.c_[
    df_all.select_dtypes(exclude=['object']).values,
    np.array(list(map(factorize, df_obj.iteritems()))).T
]
print(X_all.shape)

X_train = X_all[:num_train]
X_test = X_all[num_train:]


# Deal with categorical values
df_numeric = df_all.select_dtypes(exclude=['object'])
df_obj = df_all.select_dtypes(include=['object']).copy()

for c in df_obj:
    df_obj[c] = pd.factorize(df_obj[c])[0]

df_values = pd.concat([df_numeric, df_obj], axis=1)


# Convert to numpy values
X_all = df_values.values
print(X_all.shape)

X_train = X_all[:num_train]
X_test = X_all[num_train:]

df_columns = df_values.columns


predictions, model = runLGBM(X_train,y_train,X_test,seed_val=42)
df_sub = pd.DataFrame({'id': id_test, 'price_doc': predictions})

first_result = output_1.merge(df_sub, on="id", suffixes=('_louis', '_bruno'))
first_result["price_doc"] = np.exp( .714*np.log(first_result.price_doc_louis)) + .286*np.log(first_result.price_doc_bruno)  # multiplies out to .5 & .2
result = first_result.merge(output_2, on="id", suffixes=['_follow','_gunja'])

result["price_doc"] = np.exp(.78*np.log(result.price_doc_follow) + .22*np.log(result.price_doc_gunja))
result.drop(["price_doc_louis","price_doc_bruno","price_doc_follow","price_doc_gunja"],axis=1,inplace=True)

print(result.head())

result.to_csv('submission.csv', index = False)


#pd.DataFrame({'id': id_test, 'price_doc': np.exp(predictions)}).to_csv('submission.csv', index=False)